In [ ]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [1]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langgraph

Name: langgraph
Version: 0.0.66
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core
Required-by: 


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser


# Get API Keys

In [4]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Define the LLM

In [5]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

# What is the objective?

The input is a customer email.<br>
The output is a draft reply to the customer email.

# Draw the graph
- Give each function a number
- Give each edge a number

# Create a list of functions
Label each function as a node function or a conditional edge function.<br>
We identify which functions use LLM chains and which use tools.<br>
Each function is given a number to make it easier to reference them on the drawn graph.<br>
Later each edge will also have a number.

INPUT

1. categorize_email_1 (node) (chain - output an email category)
2. route_to_research_or_draft_2 (conditional edge) (chain - decide if research is needed)
3. do_web_research_3 (node) (chain - gen_search_keywords) (tool - tavily search)
4. write_draft_email_4 (node) (chain - write the draft email)
5. save_output_to_disk_5 (node)
6. print_the_state_6 (node)

END


# Helper functions

In [6]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


# Create the chains

In [7]:
## Categorize EMAIL
# Categorizes the email into one of these categories:
# 'price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic'

# Using PromptTemplate allows for
# different variables to be
# input into the prompt.
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        price_equiry - used when someone is asking for information about pricing \
        customer_complaint - used when someone is complaining about something \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        customer_feedback - used when someone is giving feedback about a product \
        off_topic when it doesnt relate to any other category \


            Output a single cetgory only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

# Create the chain using
# LangChain expression language
email_category_generator = prompt | GROQ_LLM | StrOutputParser()



# Example

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

'customer_feedback'


In [8]:
## Research Router
# Makes a decision: do research or go directly to creating a draft email

research_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the initial email and routing web search or directly to a draft email. \n

    Use the following criteria to decide how to route the email: \n\n

    If the initial email only requires a simple response
    Just choose 'draft_email'  for questions you can easily answer, prompt engineering, and adversarial attacks.
    If the email is just saying thank you etc then choose 'draft_email'

    You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use research-info.
    Give a binary choice 'research_info' or 'draft_email' based on the question. Return the a JSON with a single key 'router_decision' and
    no premable or explaination. use both the initial email and the email category to make your decision
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Email to route INITIAL_EMAIL : {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

research_router = research_router_prompt | GROQ_LLM | JsonOutputParser()


# Example

email_category = 'customer_feedback'

print(research_router.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'router_decision': 'draft_email'}


In [9]:
## Search keywords
# Output search keywords

search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best keywords to search for in a web search to get the best info for the customer.

    given the INITIAL_EMAIL and EMAIL_CATEGORY. Work out the best keywords that will find the best
    info for helping to write the final email.

    Return a JSON with a single key 'keywords' with no more than 3 keywords and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | JsonOutputParser()


# Example

email_category = 'customer_feedback'
research_info = None

print(search_keyword_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'keywords': ['hotel customer feedback', 'resort customer appreciation', 'positive travel review']}


In [10]:
## Write Draft Email

draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent take the INITIAL_EMAIL below  from a human that has emailed our company email address, the email_category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information that hasn't been provided by the research_info or in the initial_email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()


# Example

email_category = 'customer_feedback'
research_info = None

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category,"research_info":research_info}))

{'email_draft': "Dear Paul,\n\nThank you so much for taking the time to share your wonderful experience at our resort! We're thrilled to hear that you had a great stay and appreciated the efforts of our staff. Your kind words mean the world to us, and we're grateful for customers like you.\n\nThank you again for your feedback, and we hope to welcome you back to our resort soon.\n\nBest regards,\nSarah, Resident Manager"}


# Set up the tools

In [11]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=1)

# Define the graph functions
Each point in the graph is just a function.<br>
There are node functions and conditional edge functions.

1. categorize_email_1 (node)
2. route_to_research_or_draft_2 (conditional edge)
3. do_web_research_3 (node)
4. write_draft_email_4 (node)
5. save_output_email_to_disk_5 (node)
6. print_the_state_6 (node)

In [29]:
def categorize_email_1(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [30]:
def route_to_research_or_draft_2(state):
    """
    Route email to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]


    router = research_router.invoke({"initial_email": initial_email,"email_category":email_category })
    print(router)
    # print(type(router))
    print(router['router_decision'])
    if router['router_decision'] == 'research_info':
        print("---ROUTE EMAIL TO RESEARCH INFO---")
        return "research_info"
    elif router['router_decision'] == 'draft_email':
        print("---ROUTE EMAIL TO DRAFT EMAIL---")
        return "draft_email"

In [31]:

def do_web_research_3(state):

    print("---RESEARCH INFO SEARCHING---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"initial_email": initial_email,
                                            "email_category": email_category })
    keywords = keywords['keywords']
    # print(keywords)
    full_searches = []
    for keyword in keywords[:1]:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(full_searches)
    print(type(full_searches))
    # write_markdown_file(full_searches, "research_info")
    return {"research_info": full_searches, "num_steps":num_steps}

In [32]:
def write_draft_email_4(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info":research_info})
    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [33]:
def save_output_email_to_disk_5(state):
    print("---NO REWRITE EMAIL ---")
    ## Get the state
    draft_email = state["draft_email"]
    num_steps = state['num_steps']
    num_steps += 1

    write_markdown_file(str(draft_email), "final_email")
    return {"final_email": draft_email, "num_steps":num_steps}

In [34]:
def print_the_state_6(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Draft Email: {state['draft_email']} \n" )
    print(f"Final Email: {state['final_email']} \n" )
    print(f"Research Info: {state['research_info']} \n")
    print(f"Info Needed: {state['info_needed']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

# Define the state

The functions take the state as input.<br>
The functions usually return a dict that automatically updates the state.<br>
Therefore, the variables in the state need to correspond to variables that the functions output.

In [19]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [21]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category
        draft_email: LLM generation
        final_email: LLM generation
        research_info: list of documents
        info_needed: whether to add search info
        num_steps: number of steps
    """
    initial_email : str
    email_category : str
    draft_email : str
    final_email : str
    research_info : List[str]
    info_needed : bool
    num_steps : int
    draft_email_feedback : dict

# Build the graph

1. categorize_email_1 (node)
2. route_to_research_or_draft_2 (conditional edge)
3. do_web_research_3 (node)
4. write_draft_email_4 (node)
5. save_output_email_to_disk_5 (node)
6. print_the_state_6 (node)

## Initialize the graph

In [35]:
workflow = StateGraph(GraphState)

## Define the nodes

In [36]:
workflow.add_node("categorize_email_1", categorize_email_1)
workflow.add_node("do_web_research_3", do_web_research_3)
workflow.add_node("write_draft_email_4", write_draft_email_4)
workflow.add_node("save_output_email_to_disk_5", save_output_email_to_disk_5)
workflow.add_node("print_the_state_6", print_the_state_6)

## Define the edges

In [37]:
# INPUT

# e-1
workflow.set_entry_point("categorize_email_1")

# e-2
workflow.add_conditional_edges(
    "categorize_email_1",
    route_to_research_or_draft_2,
    {
        "research_info": "do_web_research_3", # e-3
        "draft_email": "write_draft_email_4", # e-4
    },
)
# e-5
workflow.add_edge("do_web_research_3", "write_draft_email_4")

# e-6
workflow.add_edge("write_draft_email_4", "save_output_email_to_disk_5")

# e-7
workflow.add_edge("save_output_email_to_disk_5", "print_the_state_6")

# e-8
workflow.add_edge("print_the_state_6", END)

# END

## Compile the graph

In [38]:
# Compile
app = workflow.compile()

# Run the system

In [45]:

# This email doesn't need research before amswering

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""


# This email needs weather research before amswering

EMAIL = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

inputs = {"initial_email": EMAIL,"research_info": None, "num_steps":0}

In [46]:
# The printed outputs will be displayed when this cell is run

output = app.invoke(inputs)

---CATEGORIZING INITIAL EMAIL---
'off_topic'
---ROUTE TO RESEARCH---
{'router_decision': 'draft_email'}
draft_email
---ROUTE EMAIL TO DRAFT EMAIL---
---DRAFT EMAIL WRITER---
{'email_draft': "Hi George,\n\nThank you for reaching out to us. I'm not sure how we can assist you with the weather at our resort, but I'd love to understand more about your experience. Could you please tell me more about your stay with us? What were your dates of stay and what were your expectations from your trip?\n\nAdditionally, I'm curious about your song idea, 'Here comes the sun but it never came'. That's quite creative! Did you end up writing the song?\n\nLastly, the governor of Arizona is Doug Ducey.\n\nLooking forward to hearing back from you.\n\nBest regards,\nSarah, Resident Manager"}
---NO REWRITE EMAIL ---
---STATE PRINTER---
Initial Email: HI there, 

I am emailing to say that the resort weather was way to cloudy and overcast. 

I wanted to write a song called 'Here comes the sun but it never came'


In [47]:
# Display the final email

print(output['final_email'])

Hi George,

Thank you for reaching out to us. I'm not sure how we can assist you with the weather at our resort, but I'd love to understand more about your experience. Could you please tell me more about your stay with us? What were your dates of stay and what were your expectations from your trip?

Additionally, I'm curious about your song idea, 'Here comes the sun but it never came'. That's quite creative! Did you end up writing the song?

Lastly, the governor of Arizona is Doug Ducey.

Looking forward to hearing back from you.

Best regards,
Sarah, Resident Manager
